In [2]:
import pandas as pd

In [10]:
dataset = pd.read_csv("PreTelco-Customer-Churn.csv")
dataset

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [11]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset=dataset.astype(int)
dataset

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,1,29,29,0,1,0,0,1,0,...,0,0,0,0,0,1,0,1,0,0
1,0,34,56,1889,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,2,53,108,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,1
3,0,45,42,1840,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
4,0,2,70,151,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84,1990,1,1,1,1,0,1,...,1,0,1,1,0,1,0,0,1,0
7039,0,72,103,7362,0,1,1,1,0,1,...,1,0,1,1,0,1,1,0,0,0
7040,0,11,29,346,0,1,1,0,1,0,...,0,0,0,0,0,1,0,1,0,0
7041,1,4,74,306,1,1,0,1,0,1,...,0,0,0,0,0,1,0,0,1,1


In [12]:
dataset["Churn_Yes"].value_counts()

Churn_Yes
0    5174
1    1869
Name: count, dtype: int64

In [13]:
dataset.columns.tolist()

['SeniorCitizen',
 'tenure',
 'MonthlyCharges',
 'TotalCharges',
 'gender_Male',
 'Partner_Yes',
 'Dependents_Yes',
 'PhoneService_Yes',
 'MultipleLines_No phone service',
 'MultipleLines_Yes',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes',
 'OnlineBackup_No internet service',
 'OnlineBackup_Yes',
 'DeviceProtection_No internet service',
 'DeviceProtection_Yes',
 'TechSupport_No internet service',
 'TechSupport_Yes',
 'StreamingTV_No internet service',
 'StreamingTV_Yes',
 'StreamingMovies_No internet service',
 'StreamingMovies_Yes',
 'Contract_One year',
 'Contract_Two year',
 'PaperlessBilling_Yes',
 'PaymentMethod_Credit card (automatic)',
 'PaymentMethod_Electronic check',
 'PaymentMethod_Mailed check',
 'Churn_Yes']

In [14]:
independent=dataset[['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'gender_Male', 'Partner_Yes', 'Dependents_Yes',
 'PhoneService_Yes', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No internet service',
 'TechSupport_Yes', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_One year',
 'Contract_Two year', 'PaperlessBilling_Yes', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check']]
dependent=dataset[["Churn_Yes"]]

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)

In [16]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')  
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda\envs\aidl\Lib\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [18]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print("The confusion Matrix:\n",cm)

from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print("The report:\n",clf_report)

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The confusion Matrix:
 [[1545  184]
 [ 279  340]]
The report:
               precision    recall  f1-score   support

           0       0.85      0.89      0.87      1729
           1       0.65      0.55      0.59       619

    accuracy                           0.80      2348
   macro avg       0.75      0.72      0.73      2348
weighted avg       0.79      0.80      0.80      2348

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.7972524732543607


In [19]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8383528723635858

In [20]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.113066,0.011366,0.030005,0.006490,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.796237,0.795425,0.799553,0.781937,0.810195,0.796669,0.009056,1
1,0.076960,0.022987,0.027515,0.009728,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.797167,0.794153,0.799553,0.781937,0.810195,0.796601,0.009109,4
2,0.119445,0.016160,0.031277,0.005921,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.796237,0.795425,0.799553,0.781937,0.810195,0.796669,0.009056,1
3,0.416961,0.066320,0.025081,0.006283,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.796237,0.795425,0.799553,0.781937,0.810195,0.796669,0.009056,1


In [21]:
import pickle
filename="Final_model_TCCP.sav"
pickle.dump(grid,open(filename,'wb'))

In [27]:
load_model=pickle.load(open("Final_model_TCCP.sav",'rb'))
SeniorCitizen=float(input("SeniorCitizen:"))
tenure=float(input("tenure:"))
MonthlyCharges=float(input("MonthlyCharges:"))
TotalCharges=float(input("TotalCharges:"))
gender_Male=float(input("gender_Male:"))
Partner_Yes=float(input("Partner_Yes:"))
Dependents_Yes=float(input("Dependents_Yes:"))
PhoneService_Yes=float(input("PhoneService_Yes:"))
MultipleLines_No=float(input("MultipleLines_No phone service:"))
MultipleLines_Yes=float(input("MultipleLines_Yes:"))
InternetService_Fiber=float(input("InternetService_Fiber optic:"))
InternetService_No=float(input("InternetService_No:"))
OnlineSecurity_No=float(input("OnlineSecurity_No internet service:"))
OnlineSecurity_Yes=float(input("OnlineSecurity_Yes:"))
OnlineBackup_No=float(input("OnlineBackup_No internet service:"))
OnlineBackup_Yes=float(input("OnlineBackup_Yes:"))
DeviceProtection_No=float(input("DeviceProtection_No internet service:"))
DeviceProtection_Yes=float(input("DeviceProtection_Yes:"))
TechSupport_No=float(input("TechSupport_No internet service:"))
TechSupport_Yes=float(input("TechSupport_Yes:"))
StreamingTV_No=float(input("StreamingTV_No internet service:"))
StreamingTV_Yes=float(input("StreamingTV_Yes:"))
StreamingMovies_No=float(input("StreamingMovies_No internet service:"))
StreamingMovies_Yes=float(input("StreamingMovies_Yes:"))
Contract_One=float(input("Contract_One year:"))
Contract_Two=float(input("Contract_Two year:"))
PaperlessBilling_Yes=float(input("PaperlessBilling_Yes:"))
PaymentMethod_Credit=float(input("PaymentMethod_Credit card (automatic):"))
PaymentMethod_Electronic=float(input("PaymentMethod_Electronic check:"))
PaymentMethod_Mailed=float(input("PaymentMethod_Mailed check:"))

SeniorCitizen: 0
tenure: 12
MonthlyCharges: 70.5
TotalCharges: 850.5
gender_Male: 1
Partner_Yes: 1
Dependents_Yes: 0
PhoneService_Yes: 1
MultipleLines_No phone service: 0
MultipleLines_Yes: 1
InternetService_Fiber optic: 1
InternetService_No: 0
OnlineSecurity_No internet service: 0
OnlineSecurity_Yes: 1
OnlineBackup_No internet service: 0
OnlineBackup_Yes: 1
DeviceProtection_No internet service: 0
DeviceProtection_Yes: 1
TechSupport_No internet service: 0
TechSupport_Yes: 1
StreamingTV_No internet service: 0
StreamingTV_Yes: 1
StreamingMovies_No internet service: 1
StreamingMovies_Yes: 0
Contract_One year: 1
Contract_Two year: 1
PaperlessBilling_Yes: 0
PaymentMethod_Credit card (automatic): 1
PaymentMethod_Electronic check: 0
PaymentMethod_Mailed check: 0


In [29]:
Future_Prediction=grid.predict([[SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,
                                 PhoneService_Yes,MultipleLines_No,MultipleLines_Yes,InternetService_Fiber,InternetService_No,OnlineSecurity_No,
                                 OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,
                                 TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_One,
                                 Contract_Two,PaperlessBilling_Yes,PaymentMethod_Credit,PaymentMethod_Electronic,PaymentMethod_Mailed]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


In [30]:
#So based on above input the customer can be Churn